# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [51]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import gmaps
# Import API key
from api_keys import geoapify_key

In [52]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data.head()

,City,City.1,Country,Date,Lat,Lng,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
0,0,Bariloche,AR,1681416521,-41.1456,-71.3082,11,45,59.45,3.04
1,1,Kolosovka,RU,1681416660,56.4678,73.6110,100,92,21.83,15.39
2,2,Gorgan,IR,1681416660,36.8387,54.4348,0,82,57.16,0.00
3,3,Kaitangata,NZ,1681416660,-46.2817,169.8464,74,84,52.77,12.28
4,4,Bluff,NZ,1681416661,-46.6000,168.3333,11,80,52.81,17.87


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [64]:
%%capture --no-display

# Configure the map plot

map_plot_1 = city_data.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    color = "City",
    size ='Humidity')

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [59]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_narrow=city_data[(city_data['Max Temp (F)'] < 90) &
                             (city_data['Max Temp (F)'] > 65) &
                             (city_data['Wind Speed (mph)'] < 10) &
                             (city_data['Cloudiness (%)'] < 15) &
                             (city_data['Humidity (%)'] < 50)]
city_data_narrow=city_data_narrow.dropna() # drop any rows will null values
city_data_narrow.head()

,City,City.1,Country,Date,Lat,Lng,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
5,5,Temuco,CL,1681416661,-38.7333,-72.6000,9,37,71.37,2.98
131,131,Hamilton,US,1681416620,39.1834,-84.5333,0,26,82.99,4.61
173,173,Abnūb,EG,1681416714,27.2696,31.1511,0,21,68.52,3.85
215,215,Bethel,US,1681416728,41.3712,-73.4140,0,29,89.85,8.05
237,237,Bonwapitse,BW,1681416741,-23.0766,26.7037,1,31,73.62,6.33


### Step 3: Create a new DataFrame called `hotel_df`.

In [65]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame(city_data_narrow, columns=["City.1", "Country", "Lat", "Lng", "Humidity"])

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City.1,Country,Lat,Lng,Humidity,Hotel Name
5,Temuco,CL,-38.7333,-72.6000,NaN,
131,Hamilton,US,39.1834,-84.5333,NaN,
173,Abnūb,EG,27.2696,31.1511,NaN,
215,Bethel,US,41.3712,-73.4140,NaN,
237,Bonwapitse,BW,-23.0766,26.7037,NaN,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [66]:
# Set parameters to search for a hotel
radius = 10000
params = {
   "categories":"accommodation.hotel",
    "apiKey": geoapify_key
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City.1']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
Temuco - nearest hotel: No hotel found
Hamilton - nearest hotel: No hotel found
Abnūb - nearest hotel: No hotel found
Bethel - nearest hotel: No hotel found
Bonwapitse - nearest hotel: No hotel found
Carlisle - nearest hotel: No hotel found
Bhachāu - nearest hotel: No hotel found
Carroll - nearest hotel: No hotel found
Lewiston - nearest hotel: No hotel found
Vallenar - nearest hotel: No hotel found
Sur - nearest hotel: No hotel found
Millinocket - nearest hotel: No hotel found
Turbat - nearest hotel: No hotel found
Morón - nearest hotel: No hotel found
Maïné Soroa - nearest hotel: No hotel found
Duki - nearest hotel: No hotel found
Lock Haven - nearest hotel: No hotel found
Half Moon Bay - nearest hotel: No hotel found
Ajdabiya - nearest hotel: No hotel found
Phalodi - nearest hotel: No hotel found


,City.1,Country,Lat,Lng,Humidity,Hotel Name
5,Temuco,CL,-38.7333,-72.6000,NaN,No hotel found
131,Hamilton,US,39.1834,-84.5333,NaN,No hotel found
173,Abnūb,EG,27.2696,31.1511,NaN,No hotel found
215,Bethel,US,41.3712,-73.4140,NaN,No hotel found
237,Bonwapitse,BW,-23.0766,26.7037,NaN,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [71]:
%%capture --no-display

# Configure the map plot
hotel_map = city_data.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    color = "City",
    size ='Humidity',
    hover_cols=['City.1','Humidity', 'Hotel Name', 'Country']
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,City.1,Country)